# Example: Boston Airbnb Listings

Data: https://www.kaggle.com/datasets/katerynaosadchuk/boston-airbnb-listings

**Cilj: Algoritem za napovedovanje cene**

In [3]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# to display the total number columns present in the dataset
pd.set_option('display.max_columns', None)

## Basic cleaning

In [4]:
# load
file_name = "data/boston_listings.csv"
airbnb_boston = pd.read_csv(file_name)

# clean price and cleaning_fee columns
airbnb_boston['price_usd'] = airbnb_boston['price'].str.replace("$", "").str.replace(",", "").astype("float")
airbnb_boston['cleaning_fee_usd'] = airbnb_boston['cleaning_fee'].str.replace("$", "").str.replace(",", "").astype("float")
airbnb_boston.drop(columns=["price", "cleaning_fee"], inplace=True)


In [5]:
airbnb_boston.head()

,id,name,summary,access,interaction,house_rules,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_total_listings_count,host_verifications,host_identity_verified,neighbourhood,neighbourhood_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities_dict,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,is_business_travel_ready,cancellation_policy,reviews_per_month,price_usd,cleaning_fee_usd
0,3781,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,Guests solely occupy the 1 floor apartment wit...,We sometimes travel. Always available via: mob...,"No pets, no smoking.",4804,2008-12-03,Massachusetts,within a few hours,100%,44%,t,East Boston,5,"['email', 'phone', 'reviews']",f,East Boston,East Boston,2128,42.36413,-71.02991,t,Apartment,Entire home/apt,2,1.0,1.0,0.0,Real Bed,"{'TV': 1, 'Cable TV': 1, 'Wifi': 1, 'Air condi...",11,20,20,97,16,99.0,10.0,10.0,10.0,10.0,10.0,10.0,t,NaN,f,f,super_strict_30,0.28,125.0,75.0
1,5506,**$49 Special ** Private! Minutes to center!,"Private guest room with private bath, You do n...",You get full access to the guest room with pri...,"We give guests privacy, but we are available ...",No Smoking in the Building.,8229,2009-02-19,"Boston, Massachusetts, United States",within an hour,100%,100%,t,Roxbury,2,"['email', 'phone', 'reviews', 'kba']",t,Roxbury,Roxbury,2119,42.32981,-71.09559,t,Guest suite,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{'TV': 1, 'Cable TV': 1, 'Wifi': 1, 'Air condi...",1,14,44,307,106,95.0,10.0,10.0,10.0,10.0,9.0,10.0,t,Exempt: This listing is a unit that has contra...,t,f,strict_14_with_grace_period,0.79,145.0,60.0
2,6695,$99 Special!! Home Away! Condo,"Comfortable, Fully Equipped private apartment...","Full Private apartment. 1 bedroom, kitchen, ...",NaN,No Smoking in the Building.,8229,2009-02-19,"Boston, Massachusetts, United States",within an hour,100%,100%,t,Roxbury,2,"['email', 'phone', 'reviews', 'kba']",t,Roxbury,Roxbury,2119,42.32994,-71.09351,t,Condominium,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{'TV': 1, 'Cable TV': 1, 'Wifi': 1, 'Air condi...",1,4,19,266,115,96.0,10.0,10.0,10.0,10.0,9.0,10.0,t,STR-404620,t,f,strict_14_with_grace_period,0.88,169.0,80.0
3,8789,Curved Glass Studio/1bd facing Park,"Bright, 1 bed with curved glass windows facing...",Guests have access to the full unit,I'm available for questions and/or issues.,NaN,26988,2009-07-22,"Boston, Massachusetts, United States",within a few hours,100%,83%,t,Beacon Hill,10,"['email', 'phone', 'reviews', 'jumio', 'offlin...",f,Beacon Hill,Downtown,2108,42.35919,-71.06265,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{'TV': 1, 'Cable TV': 1, 'Wifi': 1, 'Air condi...",17,42,42,290,24,93.0,9.0,9.0,10.0,10.0,10.0,9.0,t,NaN,f,f,strict_14_with_grace_period,0.35,99.0,250.0
4,10730,Bright 1bed facing Golden Dome,"Bright, spacious unit, new galley kitchen, new...",Guests have access to everything in the unit.,I'm available as needed.,"NO SMOKING, NO PETS. $100 move-in fee payable ...",26988,2009-07-22,"Boston, Massachusetts, United States",within a few hours,100%,83%,t,Beacon Hill,10,"['email', 'phone', 'reviews', 'jumio', 'offlin...",f,Beacon Hill,Downtown,2108,42.35840,-71.06185,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{'TV': 1, 'Cable TV': 1, 'Wifi': 1, 'Air condi...",28,29,29,42,31,95.0,10.0,9.0,10.0,10.0,10.0,9.0,t,NaN,f,f,strict_14_with_grace_period,0.24,150.0,250.0
